In [ ]:
# Built-in library
import json
import logging
import re
import warnings
from pathlib import Path
from pprint import pprint
from typing import Any, Optional, Union

# Standard imports
import numpy as np
import numpy.typing as npt
import pandas as pd
import polars as pl
from rich.console import Console
from rich.theme import Theme

custom_theme = Theme(
    {
        "white": "#FFFFFF",  # Bright white
        "info": "#00FF00",  # Bright green
        "warning": "#FFD700",  # Bright gold
        "error": "#FF1493",  # Deep pink
        "success": "#00FFFF",  # Cyan
        "highlight": "#FF4500",  # Orange-red
    }
)
console = Console(theme=custom_theme)

# Visualization
# import matplotlib.pyplot as plt

# NumPy settings
np.set_printoptions(precision=4)

# Pandas settings
pd.options.display.max_rows = 1_000
pd.options.display.max_columns = 1_000
pd.options.display.max_colwidth = 600

# Polars settings
pl.Config.set_fmt_str_lengths(1_000)
pl.Config.set_tbl_cols(n=1_000)

warnings.filterwarnings("ignore")

# Black code formatter (Optional)
%load_ext lab_black

# auto reload imports
%load_ext autoreload
%autoreload 2

## Tools

- A tool is a function given to the LLM which fulls a specific task.
- It should contain:
  - a text description
  - a callable function
  - a function signature (i.e. the arguments it takes)
  - a return type

In [ ]:
from dataclasses import dataclass
import inspect
from typing import Callable


@dataclass
class Tool:
    name: str
    description: str
    func: Callable
    arguments: list[tuple[str, str]]
    outputs: str

    def to_string(self) -> str:
        """Convert the Tool object to a string representation."""
        args_str = ", ".join([f"{name}: {_type}" for name, _type in self.arguments])
        return (
            f"Tool Name: {self.name},"
            f"Description: {self.description},"
            f"Arguments: {args_str}",
            f"Outputs: {self.outputs}\n",
        )

    def __call__(self, *args, **kwargs) -> Any:
        """Call the function with the provided arguments."""
        return self.func(*args, **kwargs)
    
def tool(func: Callable) -> Tool:
    """
    Decorator to create a Tool object from a function.
    """
    signature = inspect.signature(func)

In [ ]:
# Create a simple tool
def add(a: int, b: int) -> int:
    """Add two numbers."""
    return a + b